In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input
import numpy as np

# Load Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split the dataset into train and test sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the input data for the neural network
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Neural network for feature extraction
input_layer = Input(shape=(4,))  # Iris dataset has 4 features
hidden_layer = Dense(8, activation='relu', name='feature_layer')(input_layer)
output_layer = Dense(3, activation='softmax')(hidden_layer)  # 3 classes in Iris dataset

nn_model = Model(inputs=input_layer, outputs=output_layer)

# Compile and train the neural network
nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
nn_model.fit(x_train_scaled, y_train, epochs=50, batch_size=8, verbose=0)

# Create a feature extractor model
feature_extractor = Model(inputs=nn_model.input, outputs=nn_model.get_layer('feature_layer').output)

def extract_features(X):
    """Transforms input data into features using the trained NN."""
    return feature_extractor.predict(X)

# Create a pipeline for the hybrid model
hybrid_model = make_pipeline(
    FunctionTransformer(extract_features),  # Feature extraction using NN
    RandomForestClassifier(random_state=42)  # Classification using RF
)

# Train the hybrid model
hybrid_model.fit(x_train_scaled, y_train)

# Evaluate the hybrid model
y_pred = hybrid_model.predict(x_test_scaled)
print("Hybrid Model Accuracy:", accuracy_score(y_test, y_pred))


2024-12-27 15:38:05.244186: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-27 15:38:05.376980: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-27 15:38:05.379109: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-27 15:38:07.115103: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-12-27 15:38:09.896723: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Do

1/1 [==============================] - 0s 16ms/step
Hybrid Model Accuracy: 1.0
